In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df1= pd.read_csv(r"C:\Users\i\Downloads\anger-ratings-0to1.train.txt", delimiter='\t', header=None)
df1.columns = ['Id', 'tweet', 'emotion', 'score']

In [3]:
print(df1)


        Id                                              tweet emotion  score
0    10000  How the fu*k! Who the heck! moved my fridge!.....   anger  0.938
1    10001  So my Indian Uber driver just called someone t...   anger  0.896
2    10002  @DPD_UK I asked for my parcel to be delivered ...   anger  0.896
3    10003  so ef whichever butt wipe pulled the fire alar...   anger  0.896
4    10004  Don't join @BTCare they put the phone down on ...   anger  0.896
..     ...                                                ...     ...    ...
852  10852   rose incense are the best thing I've ever bought   anger  0.125
853  10853         @jaaames1993 Literally burst out laughing.   anger  0.067
854  10854           Follow up. Follow through. Be . #success   anger  0.125
855  10855  Wrinkles should merely hide where frown have b...   anger  0.125
856  10856  Love the new song I can't stop thinking about ...   anger  0.083

[857 rows x 4 columns]


In [4]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import emoji
import re

In [5]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
df1['tweet'] = df1['tweet'].apply(preprocess_tweet)

In [6]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

df1['tweet'] = df1['tweet'].apply(remove_mentions)


In [7]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text) 
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df1['tweet'] = df1['tweet'].apply(clean_text)



In [8]:
df1

,Id,tweet,emotion,score
0,10000,how the fuk who the heck moved my fridge shoul...,anger,0.938
1,10001,so my indian uber driver just called someone t...,anger,0.896
2,10002,i asked for my parcel to be delivered to a pic...,anger,0.896
3,10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896
4,10004,dont join they put the phone down on you talk ...,anger,0.896
...,...,...,...,...
852,10852,rose incense are the best thing ive ever bought,anger,0.125
853,10853,literally burst out laughing,anger,0.067
854,10854,follow up follow through be #success,anger,0.125
855,10855,wrinkles should merely hide where frown have b...,anger,0.125


In [9]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text

In [10]:
df1['tweet'] = df1['tweet'].apply(convert_emoji)

In [11]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

df1['tweet'] = df1['tweet'].apply(tokenize_tweets)

In [12]:
df1

,Id,tweet,emotion,score
0,10000,"[how, the, fuk, who, the, heck, moved, my, fri...",anger,0.938
1,10001,"[so, my, indian, uber, driver, just, called, s...",anger,0.896
2,10002,"[i, asked, for, my, parcel, to, be, delivered,...",anger,0.896
3,10003,"[so, ef, whichever, butt, wipe, pulled, the, f...",anger,0.896
4,10004,"[dont, join, they, put, the, phone, down, on, ...",anger,0.896
...,...,...,...,...
852,10852,"[rose, incense, are, the, best, thing, ive, ev...",anger,0.125
853,10853,"[literally, burst, out, laughing]",anger,0.067
854,10854,"[follow, up, follow, through, be, #success]",anger,0.125
855,10855,"[wrinkles, should, merely, hide, where, frown,...",anger,0.125


In [13]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'all',
    'about', 'above', 'after', 'again', 'against', 'ain', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't",
    'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can',
    'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't",
    'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't",
    'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how',
    'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma',
    'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no',
    'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves',
    'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn',
    "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them',
    'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until',
    'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where',
    'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you',
    "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']
def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens

df1['tweet'] = df1['tweet'].apply(remove_stopwords)

In [14]:
df1

,Id,tweet,emotion,score
0,10000,"[fuk, heck, moved, fridge, knock, landlord, do...",anger,0.938
1,10001,"[indian, uber, driver, called, someone, word, ...",anger,0.896
2,10002,"[asked, parcel, delivered, pick, store, addres...",anger,0.896
3,10003,"[ef, whichever, butt, wipe, pulled, fire, alar...",anger,0.896
4,10004,"[dont, join, put, phone, talk, rude, taking, m...",anger,0.896
...,...,...,...,...
852,10852,"[rose, incense, best, thing, ive, ever, bought]",anger,0.125
853,10853,"[literally, burst, laughing]",anger,0.067
854,10854,"[follow, follow, #success]",anger,0.125
855,10855,"[wrinkles, merely, hide, frown, mark, twain]",anger,0.125


In [15]:
import torch
from transformers import BertModel, BertTokenizer

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def extract_features(tweet):
    tokenized_text = ' '.join(tweet)
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids)
    last_hidden_states = outputs.last_hidden_state
    features = last_hidden_states.squeeze(0).numpy()
    
    return features

df1['features'] = df1['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
from sklearn.utils import shuffle
df1 = shuffle(df1)
df1

,Id,tweet,emotion,score,features
774,10774,"[focus, dont, let, others, love, define, youre...",anger,0.292,"[[-0.16542822, 0.34364292, -0.38134283, -0.027..."
745,10745,"[wolfpack, theme, trons, sting, wore, wolf, sh...",anger,0.312,"[[0.09369362, -0.03939514, -0.03917374, 0.2356..."
101,10101,"[majority, people, irritate, fuck, cba, people...",anger,0.862,"[[-0.17213908, 0.3319891, -0.037328538, -0.553..."
191,10191,"[half, epicenter, shut, police, yall, burst, t...",anger,0.625,"[[-0.124072574, 0.36686358, 0.25633517, -0.231..."
767,10767,"[thats, joke, know, incense]",anger,0.222,"[[-0.0002665827, 0.39932182, -0.053959202, 0.0..."
...,...,...,...,...,...
59,10059,"[ikr, people, still, got, grudge, reason, like...",anger,0.750,"[[0.060917467, 0.10603421, 0.18070087, 0.02237..."
761,10761,"[houston, might, lose, coach, tomorrow, midnig...",anger,0.312,"[[0.00772772, -0.0036625378, 0.317061, -0.0307..."
0,10000,"[fuk, heck, moved, fridge, knock, landlord, do...",anger,0.938,"[[-0.09296838, 0.5572394, 0.11283746, -0.31033..."
176,10176,"[girl, sitting, front, chewing, gum, like, cow...",anger,0.646,"[[-0.118008755, 0.057007827, 0.22342765, 0.076..."


In [17]:
df1.features[0].shape

(14, 768)

In [18]:

df1.head(10)

,Id,tweet,emotion,score,features
774,10774,"[focus, dont, let, others, love, define, youre...",anger,0.292,"[[-0.16542822, 0.34364292, -0.38134283, -0.027..."
745,10745,"[wolfpack, theme, trons, sting, wore, wolf, sh...",anger,0.312,"[[0.09369362, -0.03939514, -0.03917374, 0.2356..."
101,10101,"[majority, people, irritate, fuck, cba, people...",anger,0.862,"[[-0.17213908, 0.3319891, -0.037328538, -0.553..."
191,10191,"[half, epicenter, shut, police, yall, burst, t...",anger,0.625,"[[-0.124072574, 0.36686358, 0.25633517, -0.231..."
767,10767,"[thats, joke, know, incense]",anger,0.222,"[[-0.0002665827, 0.39932182, -0.053959202, 0.0..."
23,10023,"[tasers, immobilize, taser, someone, fuck, nee...",anger,0.812,"[[-0.08561629, 0.37572518, 0.14344436, -0.5390..."
387,10387,"[#epipen, public, outrage, occurs, expand, #pa...",anger,0.500,"[[-0.15601857, -0.055495482, 0.10573532, -0.06..."
776,10776,"[frown, cup, cheeks, hands, step, aside, angel...",anger,0.304,"[[-0.24261428, 0.18392316, -0.07424502, -0.218..."
753,10753,"[orange, one, #poisonous, #bitter, shouldnt, j...",anger,0.312,"[[-0.32387605, 0.12601873, -0.3613245, 0.01979..."
640,10640,"[dude, new, madden, haha]",anger,0.231,"[[-0.4428942, 0.1035945, 0.012004156, -0.16633..."


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

features = df1['features'].tolist()
padded_features = pad_sequences(features, padding='post')
padded_df = df1.copy()
padded_df['features'] = padded_features.tolist()

In [20]:
x = np.stack(padded_df['features'])

In [21]:
y = np.array(padded_df['score'])    
print("Input shape:", x.shape)
print("Output shape:", y.shape)

Input shape: (857, 39, 768)
Output shape: (857,)


In [23]:
x = np.reshape(x, (857, 39 * 768))  
y = np.reshape(y, (857,))

In [24]:
y

array([0.292, 0.312, 0.862, 0.625, 0.222, 0.812, 0.5  , 0.304, 0.312,
       0.231, 0.574, 0.458, 0.125, 0.425, 0.375, 0.333, 0.37 , 0.458,
       0.562, 0.125, 0.562, 0.864, 0.417, 0.542, 0.458, 0.479, 0.5  ,
       0.479, 0.375, 0.75 , 0.729, 0.583, 0.417, 0.75 , 0.354, 0.417,
       0.479, 0.75 , 0.583, 0.479, 0.521, 0.271, 0.553, 0.938, 0.354,
       0.625, 0.69 , 0.417, 0.188, 0.083, 0.458, 0.792, 0.479, 0.542,
       0.708, 0.863, 0.458, 0.583, 0.146, 0.229, 0.479, 0.438, 0.438,
       0.688, 0.571, 0.312, 0.375, 0.375, 0.75 , 0.5  , 0.521, 0.396,
       0.396, 0.75 , 0.646, 0.625, 0.479, 0.396, 0.396, 0.479, 0.519,
       0.667, 0.625, 0.521, 0.491, 0.229, 0.208, 0.312, 0.517, 0.5  ,
       0.25 , 0.458, 0.188, 0.5  , 0.462, 0.604, 0.521, 0.333, 0.562,
       0.328, 0.067, 0.479, 0.625, 0.208, 0.521, 0.375, 0.517, 0.296,
       0.479, 0.472, 0.229, 0.792, 0.562, 0.466, 0.271, 0.146, 0.542,
       0.354, 0.479, 0.667, 0.208, 0.292, 0.188, 0.583, 0.844, 0.143,
       0.875, 0.479,

In [25]:
X=x.copy()  
Y=y.copy()  

In [26]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

alpha = 2000
model = Ridge(alpha=alpha)
model.fit(X, Y)

Ridge(alpha=2000)

In [27]:
y_pred = model.predict(X)
mse = mean_squared_error(Y, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.01872813812461101


In [28]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y, y_pred)

print("Mean Absolute Error:", mae)

Mean Absolute Error: 0.10816342059710127


In [29]:
import joblib
save_path = 'regression_model.pkl'
joblib.dump(model, save_path)


['regression_model.pkl']

In [30]:
df2= pd.read_csv(r"C:\Users\i\Downloads\anger-ratings-0to1.dev.gold.txt", delimiter='\t', header=None)
df2.columns = ['Id', 'tweet', 'emotion', 'score']

In [31]:
df2

,Id,tweet,emotion,score
0,10857,@ZubairSabirPTI pls dont insult the word 'Molna',anger,0.479
1,10858,@ArcticFantasy I would have almost took offens...,anger,0.458
2,10859,@IllinoisLoyalty that Rutgers game was an abom...,anger,0.562
3,10860,@CozanGaming that's what lisa asked before she...,anger,0.500
4,10861,Sometimes I get mad over something so minuscul...,anger,0.708
...,...,...,...,...
79,10936,@Jen_ny69 People will always get offended ever...,anger,0.562
80,10937,@gayla_weeks1 I try not to let my anger seep i...,anger,0.625
81,10938,I hope my hustle don't offend nobody,anger,0.292
82,10939,"Just watched Django Unchained, Other people ma...",anger,0.229


In [32]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
df2['tweet'] = df2['tweet'].apply(preprocess_tweet)

In [33]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

df2['tweet'] = df2['tweet'].apply(remove_mentions)


In [34]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text)  
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df2['tweet'] = df2['tweet'].apply(clean_text)


In [35]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text
df2['tweet'] = df2['tweet'].apply(convert_emoji)

In [36]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

df2['tweet'] = df2['tweet'].apply(tokenize_tweets)

In [37]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'all',
    'about', 'above', 'after', 'again', 'against', 'ain', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't",
    'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can',
    'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't",
    'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't",
    'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how',
    'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma',
    'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no',
    'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves',
    'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn',
    "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them',
    'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until',
    'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where',
    'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you',
    "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']


def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens
df2['tweet'] = df2['tweet'].apply(remove_stopwords)

In [38]:
import torch
from transformers import BertModel, BertTokenizer

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


def extract_features(tweet):
    
    tokenized_text = ' '.join(tweet)
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids)
    last_hidden_states = outputs.last_hidden_state
    features = last_hidden_states.squeeze(0).numpy()
    
    return features
df2['features'] = df2['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

features2 = df2['features'].tolist()
padded_features2 = pad_sequences(features2, padding='post')
padded_df2 = df2.copy()
padded_df2['features'] = padded_features2.tolist()

In [40]:
X = np.stack(padded_df2['features'])
print('Input feature shape:', X.shape)

Input feature shape: (84, 42, 768)


In [41]:
print('Input feature shape:', X.shape)

Input feature shape: (84, 42, 768)


In [44]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length=39
X_=pad_sequences(X,maxlen=max_sequence_length,padding="post",truncating='post')

In [45]:
X_ = np.reshape(X_, (84, 39 * 768))  

In [46]:
saved_model_path = 'regression_model.pkl'
loaded_model = joblib.load(saved_model_path)

In [47]:
predictions = loaded_model.predict(X_)
predictions

array([0.49842736, 0.46203439, 0.53270571, 0.49154141, 0.51946799,
       0.49900371, 0.52337554, 0.50929572, 0.54080388, 0.49334245,
       0.50756418, 0.50020949, 0.50724266, 0.53093603, 0.50609313,
       0.52949005, 0.53945058, 0.50126373, 0.47355087, 0.48728093,
       0.49374787, 0.49815871, 0.4703646 , 0.51879091, 0.51026489,
       0.48904454, 0.48825501, 0.50775732, 0.5571534 , 0.56283485,
       0.47003905, 0.49016105, 0.48620832, 0.46474187, 0.51269573,
       0.51122319, 0.50963272, 0.47945398, 0.5067111 , 0.49413188,
       0.46216495, 0.49043235, 0.43621519, 0.43578475, 0.53542   ,
       0.5537343 , 0.49283092, 0.4785767 , 0.54021132, 0.49563876,
       0.47192046, 0.49303141, 0.46593574, 0.50834858, 0.50291306,
       0.51509979, 0.4609621 , 0.49610944, 0.52690274, 0.49458277,
       0.49568274, 0.48274893, 0.50553983, 0.51949117, 0.47356065,
       0.47066734, 0.48628545, 0.46517613, 0.46517613, 0.49919687,
       0.46144758, 0.47798402, 0.50057393, 0.50495191, 0.46030

In [48]:
df2['predict']=pd.DataFrame(predictions)

In [49]:
mse=mean_squared_error(df2['score'],df2['predict'])
print("mean_square_error:",mse)

mean_square_error: 0.022288857486363632


In [50]:
mae = mean_absolute_error(df2['score'],df2['predict'])
print("Mean Absolute Error:", mae)


Mean Absolute Error: 0.11620193283453203


In [51]:
df2

,Id,tweet,emotion,score,features,predict
0,10857,"[pls, dont, insult, word, molna]",anger,0.479,"[[-0.16410865, 0.59082854, -0.29882985, -0.219...",0.498427
1,10858,"[would, almost, took, offense, actually, snapped]",anger,0.458,"[[0.016600985, 0.094489805, 0.05158711, 0.0325...",0.462034
2,10859,"[rutgers, game, abomination, affront, god, man...",anger,0.562,"[[-0.05811101, 0.47538888, -0.17922018, -0.149...",0.532706
3,10860,"[thats, lisa, asked, started, raging, call, heh]",anger,0.500,"[[-0.23008505, 0.32468075, 0.22509159, -0.2303...",0.491541
4,10861,"[sometimes, get, mad, something, minuscule, tr...",anger,0.708,"[[0.26804066, -0.19931054, 0.14129303, -0.0347...",0.519468
...,...,...,...,...,...,...
79,10936,"[people, always, get, offended, everyones, sit...",anger,0.562,"[[0.28602043, 0.51173747, -0.058281485, 0.2042...",0.498812
80,10937,"[try, let, anger, seep, reviews, resent, time,...",anger,0.625,"[[0.1119683, -0.38341117, -0.030351479, -0.021...",0.523604
81,10938,"[hope, hustle, dont, offend, nobody]",anger,0.292,"[[-0.023534112, 0.5019074, -0.48025265, -0.017...",0.462374
82,10939,"[watched, django, unchained, people, may, frow...",anger,0.229,"[[0.15952902, 0.13285297, 0.2045169, 0.0801301...",0.503383


In [52]:
df3= pd.read_csv(r"C:\Users\i\Downloads\anger-ratings-0to1.test.target.txt", delimiter='\t', header=None)
df3.columns = ['Id', 'tweet', 'emotion', 'score']

In [53]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
df3['tweet'] = df3['tweet'].apply(preprocess_tweet)

In [54]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

df3['tweet'] = df3['tweet'].apply(remove_mentions)

In [55]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text)  
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df3['tweet'] = df3['tweet'].apply(clean_text)


In [56]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text
df3['tweet'] = df3['tweet'].apply(convert_emoji)

In [57]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

df3['tweet'] = df3['tweet'].apply(tokenize_tweets)

In [58]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'all',
    'about', 'above', 'after', 'again', 'against', 'ain', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't",
    'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can',
    'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't",
    'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't",
    'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how',
    'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma',
    'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no',
    'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves',
    'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn',
    "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them',
    'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until',
    'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where',
    'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you',
    "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']


def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens
df3['tweet'] = df3['tweet'].apply(remove_stopwords)

In [59]:
import torch
from transformers import BertModel, BertTokenizer

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


def extract_features(tweet):
    
    tokenized_text = ' '.join(tweet)
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids)
    last_hidden_states = outputs.last_hidden_state
    features = last_hidden_states.squeeze(0).numpy()
    
    return features
df3['features'] = df3['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [61]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

features3 = df3['features'].tolist()
padded_features3 = pad_sequences(features3, padding='post')
padded_df3 = df3.copy()
padded_df3['features'] = padded_features3.tolist()

In [62]:
X = np.stack(padded_df3['features'])
print('Input feature shape:', X.shape)

Input feature shape: (760, 43, 768)


In [66]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length=39
X_=pad_sequences(X,maxlen=max_sequence_length,padding="post",truncating='post')

In [67]:
X_ = np.reshape(X_, (760, 39 * 768))  

In [68]:
saved_model_path = 'regression_model.pkl'
loaded_model = joblib.load(saved_model_path)

In [69]:
predicttest = loaded_model.predict(X_)
predicttest

array([0.52533331, 0.44690213, 0.5098315 , 0.480763  , 0.50630038,
       0.52030501, 0.46563691, 0.50365541, 0.4819241 , 0.4390828 ,
       0.47887386, 0.51775205, 0.46161318, 0.53332154, 0.50154011,
       0.43225537, 0.45251208, 0.49180472, 0.49995856, 0.50379784,
       0.47390469, 0.48423737, 0.54341471, 0.49623301, 0.50899592,
       0.46581053, 0.46323017, 0.51584305, 0.50587553, 0.47632581,
       0.45678251, 0.52166921, 0.53935576, 0.52754997, 0.51392598,
       0.47453121, 0.50854318, 0.52659933, 0.44910776, 0.56092853,
       0.57932237, 0.48773483, 0.52134678, 0.57392528, 0.52223061,
       0.52313121, 0.50651197, 0.49360887, 0.48583735, 0.45523913,
       0.45543686, 0.48453976, 0.4880368 , 0.51375376, 0.52473707,
       0.41667003, 0.42019073, 0.50541617, 0.49828903, 0.49014918,
       0.50489872, 0.46513667, 0.49689789, 0.52662026, 0.50512586,
       0.48088079, 0.4592799 , 0.43297746, 0.50578651, 0.47916262,
       0.51760875, 0.49159447, 0.47466437, 0.43710115, 0.56649

In [70]:
df3['predicttest']=pd.DataFrame(predicttest)

In [71]:
df3=df3.drop(['score','features'],axis=1)

In [72]:
df3

,Id,tweet,emotion,predicttest
0,10941,"[point, today, someone, says, something, remot...",anger,0.525333
1,10942,"[game, day, minus, #relentless]",anger,0.446902
2,10943,"[game, pissed, game, year, blood, boiling, tim...",anger,0.509832
3,10944,"[ive, found, candice, candace, pout, likes]",anger,0.480763
4,10945,"[cant, come, muma, th, tweets, #soreloser]",anger,0.506300
...,...,...,...,...
755,11696,"[supposed, animosity, bullshit, con, iranians]",anger,0.494055
756,11697,"[byus, offense, score, vs, wvu]",anger,0.489587
757,11698,"[id, love, gyimah, action, coach, holding, gru...",anger,0.485663
758,11699,"[forgiving, means, operating, gods, spirit, am...",anger,0.501155
